### 1. 索引有哪些数据结构？
1. 哈希索引
    
    使用哈希表作为索引结构，MySQL 中只有 Memory 引擎显示支持哈希索引，查询速度最快。InnoDB有一个功能叫自适应哈希，当其注意到部分索引值使用频繁，会对其在B+索引的基础上再建一个哈希索引（用户无法控制或配置，但可以选择关闭）

哈希索引的限制：

1. 哈希索引存储的是行指针，而不是值，这样每次一定需要回表
2. 哈希索引不是按照索引值的顺序进行排列，因此无法用于排序
3. 哈希索引不支持部分索引列匹配查找
4. 哈希索引无法进行范围查找（区间，大于，小于等），只支持等值比较查询
5. 哈希冲突问题（不同key的哈希值一致）

如何解决哈希冲突：

1. 拉链法，每个哈希值后跟一个链表，在链表中存储实际的值，但是链表过长则会降低效率
2. 线性探测法，如果哈希冲突，就线性向下探测，在下一个空位填充数据；查找的时候，如果线性探测遇到空位，说明链表中无此数据。删除操作比较麻烦，简单的置为空会使得后续的索引不可达，因此需要将后续的键重新插入。

B+树索引

    为了更快的查找，树形结构有着先天性的优势，B树结构能够降低树的层数，而B+树只在叶子结点存储完整的值，通过链表进行相连，这样有利于范围查询，排序等操作。此外，由于 InnoDB 在访问行时需要加锁，索引能减少访问次数，减少锁的数量。

支持：

1. 全值匹配
2. 匹配最左前缀（只使用索引的前面几列
3. 匹配范围值
4. 精确匹配前面几列并范围匹配后面某一列
5. 覆盖索引

限制：

1. 如果不是按照最左列开始查找，则无法使用索引，如索引(a, b, c)，可以匹配(a), (a, b), (a, b, c)，不可以匹配 (a, c), (b, c)
2. 如果某列进行范围查找，后续的列无法使用索引，如 (a, b, c), 对 b 进行范围查询，则 c 无法使用索引。

因此，合适的索引列顺序非常重要。

### 2. 索引有哪些类型？

#### 2.1 多列索引

    由多个列组成的索引，什么时候需要？

    1. 频繁进行某几个列的相交或联合操作
    2. EXPALAIN 加 SQL 语句，经常看到索引合并
    3. 多列索引遵循最左匹配原则

#### 2.2 覆盖索引

    如果一个索引就包含了索引需要查询的字段值，这就称为覆盖索引。

#### 2.3 聚簇索引

    聚簇索引就是在叶节点存放一整行记录，InnoDB使用 B+树实现聚簇索引。

    优点：

    1. 把相关数据存储在一起
    2. 数据访问更快
    3. 覆盖索引

    缺点：

    1. 插入操作可能需要页分裂
    2. 导致全表扫描变慢，尤其是行比较稀疏
    3. 二级索引需要回表（非覆盖索引的情况下）


#### 2.4 二级索引

    除了主键索引外的索引叫做二级索引，二级索引存储的是主键列指针，需要根据主键再回表查询


### 3. MySQL 的逻辑架构？

    客户端 ----> 查询缓存 ----> 解析器 ----> 预处理器
                                            I
                                            I
                                            V
                                         查询优化器
                                            I
                                            I
                                            V
                                         查询执行引擎

    1. 客户端发送请求
    2. 服务器查询缓存，命中则返回结果
    3. 对 SQL 进行解析和预处理，再由优化器生成执行计划。（同一查询可能有多种执行计划，MySQL使用基于成本的优化器预测成本并选择成本最小的一个。
    4. 调用存储引擎API执行计划


### 4. 事务的特点？隔离级别？

    事务特点 - ACID：

    1. Atomicity，原子性
    
    要么全部执行，提交成功，要么不执行，全部失败回滚

    2. Consistency，一致性
    
    总是从一个一致的状态转移到另一个一致的状态

    3. Isolation，隔离性
    
    分为四个隔离级别：
    
        1. 未提交读，事务可以看到其他事务未提交的修改，这称之为脏读
        2. 提交读，只能看到其它事务提交的修改，因此在一个事务中，同一查询的多次执行，得到的结果可能不一样，这称之为不可重复读。
        3. 可重复读，保证上述问题，但是对于范围查询，其它事务插入了新的记录，就会出现不一样的结果，这称之为幻读。
        4. 可串行化，事务只能串行执行。

    4. Durability，持久性
    
    事务一旦提交，所做的修改就会永久保存到数据库。
    

### 5. MySQL 的事务是如何实现的？

    MySQL 的事务通过 Redo Log（D 持久性） ，Undo Log（A 原子性和 C 一致性） 和 加锁（I 隔离性） 来实现的。

    日志分为物理日志和逻辑日志，物理日志存储被修改的值，逻辑日志存储执行的SQL语句等。
    
    Redo Log，即重做日志，是物理日志，如事务成功提交，但数据未能完整写入，MySQL 通过它来恢复数据。它由两部分组成：

    1. Redo Log Buffer 重做日志缓冲
    2. Redo Log File   重做日志文件
    
    InnoDB 在事务提交时，会先将该事务的所有日志写入到 Redo 日志文件中，待事务成功提交才算整个事务操作完成。
    在每次将 Redo Log Buffer 写入 Redo Log File 后，都需要调用一次 FSync 操作，因为重做日志缓冲只是把内容先写入操作系统的缓冲系统中，并没有确保直接写入到磁盘上，所以必须进行一次 Fsync 操作，确保持久化。因此，磁盘的性能在一定程度上也决定了事务提交的性能。
    
    LSN：日志序列号，Checkpoint：上次完成的序列号。恢复时检查 Checkpoint 来选择需要重做的日志。例如，当数据库在 Checkpoint 的 LSN 为1000时发生宕机，恢复操作仅恢复 LSN 1000以后的日志。
    
    
    Undo log，即撤销日志，是逻辑日志，它可以实现如下两个功能：
    
    1. 事务回滚
    2. MVCC

    可以认为当 Delete 一条记录时，Undo Log 中会记录一条对应的 Insert 记录，反之亦然，当 Update 一条记录时，它记录一条对应相反的 Update记录。

    当执行回滚时，就可以从 Undo Log 中的逻辑记录读取到相应的内容并进行回滚。有时候应用到行版本控制的时候，也是通过 Undo Log来实现的：当读取的某一行被其他事务锁定时，它可以从 Undo Log中分析出该行记录以前的数据是什么，从而提供该行版本信息，帮助用户实现一致性非锁定读取。

### 6. MVCC？ 原理？

    MVCC 是多版本并发控制，是处理并行事务的。MVCC 给每一行添加了两个隐藏列（最后修改的版本号，删除的版本号）。每开始一个事务，系统版本号递增，事务开始时刻的版本号就会作为事务的版本号：
    
    1. 查询：早于当前版本创建的数据，或者删除版本大于当前版本号的数据
    2. 插入：采用当前版本作为行版本号
    3. 删除：采用当前版本作为行删除标识
    4. 修改：采用当前版本作为行版本号
    
    这样大部分的读操作不需要加锁，事务通过 Undo Log 分析之前的数据，避免脏读和幻读。

### 7. 复制？备份？

复制分为基于语句的和基于行的。

1. 基于语句的，将执行的 SQL记录下来，在另一个地方重新执行，但是如基于时间的语句，可能会得到不一样的结果，所以很少见。
2. 基于行的，将实际数据记录在二进制日志中。

复制是为了备库，可以承担读请求，实现负载均衡，故障切换。


### 8. 主从复制原理？配置？
    
    主从复制基于Binary Log（主库，二进制日志） 和 Relay Log（备库，中继日志）。

    1. 主库在每次准备提交事物完成数据库更新前，先记录二进制日志，然后再提交事务。MySQL 会按照事务提交顺序来记录日志。

    2. 备库会将主库的二进制日志复制到本地的中继日志中。首先，备库会先启动 一个 IO 工作线程，和主库建立一个普通的客户端连接，并在主库启动一个特殊的 binlog dump （二进制转储）线程，如果该线程追赶上了主库，它将进入睡眠状态，直到主库有新的事件产生通知它，此时这个线程会被唤醒，备库 I/O线程会将接收到的事件记录到中继日志中。

    3. 备库的 SQL 线程执行最后一步，该线程从中继日志中读取事件并且在备库执行，SQL线程也可以根据配置选项来决定是否将修改写入其自己的二进制日志中。
    
    配置：
    
    1. 主库上创建具有复制权限的用户，二进制日志文件
    2. 备库上使用 CHANGE MASTER TO 配置主库的用户密码，对应的二进制日志文件等
    3. START SLAVE;

### 9. 如何查看SQL使用了哪些索引

EXPLAIN + SQL

```python
mysql> explain select * from subject where id = 1 \G
******************************************************
           id: 1
  select_type: SIMPLE
        table: subject
   partitions: NULL
         type: const
possible_keys: PRIMARY
          key: PRIMARY
      key_len: 4
          ref: const
         rows: 1
     filtered: 100.00
        Extra: NULL
******************************************************
```

#### 9.1 字段

| id   | Columns       | JSON Name     | Meaning                                           |
| :--- | :------------ | :------------ | :------------------------------------------------ |
| 1    | id            | select_id     | 每个select子句的标识id                            |
| 2    | select_type   | None          | select语句的类型                                  |
| 3    | table         | table_name    | 当前表名                                          |
| 4    | partitions    | partitions    | 匹配的分区                                        |
| 5    | type          | access_type   | 当前表内访问方式 join type                        |
| 6    | possible_keys | possible_keys | 可能使用到的索引                                  |
| 7    | key           | key           | 经过优化器评估最终使用的索引                      |
| 8    | key_len       | key_length    | 使用到的索引长度                                  |
| 9    | ref           | ref           | 引用到的上一个表的列                              |
| 10   | rows          | rows          | rows_examined，要得到最终记录索要扫描经过的记录数 |
| 11   | filtered      | filtered      | 按表条件过滤行的百分比                            |
| 12   | Extra         | None          | 额外的信息说明                                    |

#### 9.2 select_type

| id   | select_type value    | JSON name                  | Meaning                                                      |
| :--- | :------------------- | :------------------------- | :----------------------------------------------------------- |
| 1    | SIMPLE               | None                       | 简单的SELECT语句（不包括UNION操作或子查询操作）              |
| 2    | PRIMARY              | None                       | PRIMARY：查询中最外层的SELECT（如两表做UNION或者存在子查询的外层的表操作为PRIMARY，内层的操作为UNION） |
| 3    | UNION                | None                       | UNION：UNION操作中，查询中处于内层的SELECT（内层的SELECT语句与外层的SELECT语句没有依赖关系） |
| 4    | DEPENDENT UNION      | dependent(true)            | DEPENDENT UNION：UNION操作中，查询中处于内层的SELECT（内层的SELECT语句与外层的SELECT语句有依赖关系） |
| 5    | UNIOIN RESULT        | union_result               | UNION RESULT：UNION操作的结果，id值通常为NULL                |
| 6    | SUBQUERY             | None                       | SUBQUERY：子查询中首个SELECT（如果有多个子查询存在）         |
| 7    | DEPENDENT SUBQUERY   | dependent(true)            | DEPENDENT SUBQUERY：子查询中首个SELECT，但依赖于外层的表（如果有多个子查询存在） |
| 8    | DERIVED              | None                       | DERIVED：被驱动的SELECT子查询（子查询位于FROM子句）          |
| 9    | MATERIALIZED         | materialized_form_subquery | MATERIALIZED：被物化的子查询                                 |
| 10   | UNCACHEABLE SUBQUERY | cacheable(false)           | UNCACHEABLE SUBQUERY：对于外层的主表，子查询不可被物化，每次都需要计算（耗时操作） |
| 11   | UNCACHEABLE UNION    | cacheable(false)           | UNCACHEABLE UNION：UNION操作中，内层的不可被物化的子查询（类似于UNCACHEABLE SUBQUERY） |

#### 9.3 type
当前表内访问方式, 性能由好到坏排序：

| id   | type value      | Meaning                                                      |
| :--- | :-------------- | :----------------------------------------------------------- |
| 1    | system          | 表中只有一行                                                 |
| 2    | const           | 单表中最多有一个匹配行，primary key 或者 unique index的检索  |
| 3    | eq_ref          | 多表连接中被驱动表的连接列上有primary key或者unique index的检索 |
| 4    | ref             | 与eq_ref类似，但不是使用primary key或者unique index，而是普通索引。也可以是单表上non-unique索引检索 |
| 5    | fulltext        | 使用FULLTEXT索引执行连接                                     |
| 6    | ref_or_null     | 与ref类似，区别在于条件中包含对NULL的查询                    |
| 7    | index_merge     | 索引合并优化，利用一个表里的N个索引查询,key_len表示这些索引键的和最长长度。 |
| 8    | unique_subquery | in的后面是一个查询primary key\unique字段的子查询             |
| 9    | index_subquery  | in的后面是一个查询普通index字段的子查询                      |
| 10   | range           | 单表索引中的范围查询,使用索引查询出单个表中的一些行数据。ref列会变为null |
| 11   | index           | 等于ALL。它有两种情况：(1)覆盖索引 (2)用索引的顺序做一个全表扫描。 |
| 12   | all             | 全表扫描                                                     |


#### 9.4  Extra

Extra是对执行计划的额外说明，包含重要信息。

例如：

| id   | type value                                          | Meaning                                                      |
| :--- | :-------------------------------------------------- | :----------------------------------------------------------- |
| 1    | const row not found                                 | 所要查询的表为空                                             |
| 2    | Distinct                                            | mysql正在查询distinct值，因此当它每查到一个distinct值之后就会停止当前组的搜索，去查询下一个值 |
| 3    | Impossible WHERE                                    | where条件总为false，表里没有满足条件的记录                   |
| 4    | Impossible WHERE noticed after reading const tables | 在优化器评估了const表之后，发现where条件均不满足             |
| 5    | no matching row in const table                      | 当前join的表为const表，不能匹配                              |
| 6    | Not exists                                          | 优化器发现内表记录不可能满足where条件                        |
| 7    | Select tables optimized away                        | 在没有group by子句时，对于MyISAM的select count(\*)操作，或者当对于min(),max()的操作可以利用索引优化，优化器发现只会返回一行。 |
| 8    | Using filesort                                      | 使用filesort来进行order by操作                               |
| 9    | Using index                                         | 覆盖索引                                                     |
| 10   | Using index for group-by                            | 对于group by列或者distinct列，可以利用索引检索出数据，而不需要去表里查数据、分组、排序、去重等等 |
| 11   | Using join buffer                                   | 之前的表连接在nested loop之后放进join buffer，再来和本表进行join。适用于本表的访问type为range，index或all |
| 12   | Using sort_union,using union,using intersect        | index_merge的三种情况                                        |
| 13   | Using temporary                                     | 使用了临时表来存储中间结果集，适用于group by，distinct，或order by列为不同表的列。 |
| 14   | Using where                                         | 在存储引擎层检索出记录后，在server利用where条件进行过滤，并返回给客户端 |

### 10. 如何优化？

1. 延迟关联，在查询的第一阶段使用覆盖索引查询对应的索引，再根据索引查询想要的值。例如LIMIT 几万，这时直接查询涉及多次页置换，
2. 避免重复索引，避免不重复的冗余索引
3. CHECK TABLE，REPAIR TABLE 查询修复损坏的表，OPTIMIZE TABLE 优化表的数据存储
4. 仅返回所需要的列
5. 大查询改为小查询，如一次删除大量数据会很慢，分成多次查询进行执行就可以改善问题
6. 分解关联查询，提升缓存效率，在应用层做关联，提升可扩展性
7. WHERE IN + 子查询的效率很差，改写为 WHERE EXISTS 更好
8. A UNION B LIMIT 20 的效果不如 A LIMIT 20 UNION ALL B LIMIT 20.
9. UNION 和 UNION ALL 的区别是, UNION 会自动压缩多个结果集合中的重复结果，而 UNION ALL 则将所有的结果全部显示出来，不管是不是重复。UNION 通过临时表进行关联，除非确实需要去重，否则就一定需要 UNION ALL。

### 11.  乐观锁，悲观锁，表锁，行锁

1. 乐观和悲观是对于并发事件的态度，乐观锁认为，只有提交数据改动的时候才需要加锁，而悲观锁在对数据进行操作前就进行加锁。
2. 表锁和行锁是不同粒度的锁。

### 12. 分布式事务如何实现？

    分布式事务是指事务的参与者、支持事务的服务器、资源服务器以及事务管理器分别位于不同的分布式系统的不同节点之上。


#### 12.1 一些概念

参考 [分布式事务，这一篇就够了](https://xiaomi-info.github.io/2020/01/02/distributed-transaction/)

1. 强一致性

    任何一次读都能读到某个数据的最近一次写的数据。系统中的所有进程，看到的操作顺序，都和全局时钟下的顺序一致。简言之，在任意时刻，所有节点中的数据是一样的。

2. 弱一致性

    数据更新后，如果能容忍后续的访问只能访问到部分或者全部访问不到，则是弱一致性。

3. 最终一致性

    不保证在任意时刻任意节点上的同一份数据都是相同的，但是随着时间的迁移，不同节点上的同一份数据总是在向趋同的方向变化。简单说，就是在一段时间后，节点间的数据会最终达到一致状态。
    
    
    转账是最经典那的分布式事务场景，假设用户 A 使用银行 app 发起一笔跨行转账给用户 B，银行系统首先扣掉用户 A 的钱，然后增加用户 B 账户中的余额。此时就会出现 2 种异常情况：1. 用户 A 的账户扣款成功，用户 B 账户余额增加失败 2. 用户 A 账户扣款失败，用户 B 账户余额增加成功。对于银行系统来说，以上 2 种情况都是不允许发生，此时就需要分布式事务来保证转账操作的成功。
    
    
#### 12.2 分布式事务的解决方案

* 两阶段提交/XA

    两阶段提交，顾名思义就是要分两步提交。存在一个负责协调各个本地资源管理器的事务管理器，本地资源管理器一般是由数据库实现，事务管理器在第一阶段的时候询问各个资源管理器是否都就绪？如果收到每个资源的回复都是 yes，则在第二阶段提交事务，如果其中任意一个资源的回复是 no, 则回滚事务。
    
    大致的流程：

    1. 第一阶段（prepare）：事务管理器向所有本地资源管理器发起请求，询问是否是 ready 状态，所有参与者都将本事务能否成功的信息反馈发给协调者；
    2. 第二阶段 (commit/rollback)：事务管理器根据所有本地资源管理器的反馈，通知所有本地资源管理器，步调一致地在所有分支上提交或者回滚。
    
    存在的问题：

    * 同步阻塞：当参与事务者存在占用公共资源的情况，其中一个占用了资源，其他事务参与者就只能阻塞等待资源释放，处于阻塞状态。

    * 单点故障：一旦事务管理器出现故障，整个系统不可用

    * 数据不一致：在阶段二，如果事务管理器只发送了部分 commit 消息，此时网络发生异常，那么只有部分参与者接收到 commit 消息，也就是说只有部分参与者提交了事务，使得系统数据不一致。

    * 不确定性：当协事务管理器发送 commit 之后，并且此时只有一个参与者收到了 commit，那么当该参与者与事务管理器同时宕机之后，重新选举的事务管理器无法确定该条消息是否提交成功。
    

### 13. SQL

1. UNION 和 UNION ALL 的区别是, UNION 会自动压缩多个结果集合中的重复结果，而 UNION ALL 则将所有的结果全部显示出来，不管是不是重复。
2. 

    ```sql
    INSERT INTO table_name ( field1, field2,...fieldN )
                           VALUES
                           ( value1, value2,...valueN );               
    ```


    ```sql
    DELETE FROM table_name [WHERE Clause]
    ```

    ```sql
    UPDATE table_name SET field1=new-value1, field2=new-value2 [WHERE Clause]
    ```
